In [123]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



## Import Modules

In [124]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import librosa
import librosa.display
from IPython.display import Audio
import warnings
warnings.filterwarnings('ignore')

## Load the Dataset

In [125]:
#creating two empty lists
paths = []
labels = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        paths.append(os.path.join(dirname, filename))
        label = filename.split('_')[-1]
        label = label.split('.')[0]
        labels.append(label.lower())
print('Dataset is Loaded')

In [126]:
paths[:5]

In [127]:
labels[:5]

In [128]:
## Create a dataframe
df = pd.DataFrame()
df['speech'] = paths
df['label'] = labels
df.head()

In [129]:
df['label'].value_counts()

In [173]:
df['label'].unique()

## Feature Extraction

In [130]:
def extract_mfcc(filename):
    y, sr = librosa.load(filename, duration=3, offset=0.5)
    #duration used to uniform the time length of each audio
    #sr is sampling rate
    mfcc = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T, axis=0)
    return mfcc

In [214]:
df['speech'][0]

In [131]:
extract_mfcc(df['speech'][0])

In [132]:
X_mfcc = df['speech'].apply(lambda x: extract_mfcc(x))

In [133]:
X_mfcc

In [134]:
#to create a numpy array
X = [x for x in X_mfcc]
X = np.array(X)
X.shape

In [141]:
## input split whihc is accepted by lstm model
X = np.expand_dims(X, -1)
X.shape

#sample,no of features

In [149]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
y = enc.fit_transform(df[['label']])

In [150]:

y = y.toarray()


In [151]:
y.shape

In [157]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
y_act= label_encoder.fit_transform(df['label'])
 
print(y_act)

In [181]:
label_encoder.classes_

## Create the LSTM Model

In [283]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

model = Sequential([
    LSTM(123, return_sequences=False, input_shape=(40,1)),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(7, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [284]:
# Train the model
history = model.fit(X, y, validation_split=0.3, epochs=100, batch_size=512, shuffle=True)

In [275]:
predict_y=model.predict(X) 
yhat=np.argmax(predict_y,axis=1)

In [276]:
predict_y

In [277]:
yhat

## Plot the results

In [278]:
import sklearn.metrics

r = sklearn.metrics.confusion_matrix(y_act, yhat)
print(r)

In [164]:
epochs = list(range(100))
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

plt.plot(epochs, acc, label='train accuracy')
plt.plot(epochs, val_acc, label='val accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [165]:
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.plot(epochs, loss, label='train loss')
plt.plot(epochs, val_loss, label='val loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

**REAL TIME EXAMPLE**


In [267]:
#path = '/kaggle/input/toronto-emotional-speech-set-tess/TESS Toronto emotional speech set data/YAF_fear/YAF_home_fear.wav'
path='../input/toronto-emotional-speech-set-tess/TESS Toronto emotional speech set data/OAF_Sad/OAF_base_sad.wav'
extract_mfcc(path)

In [268]:
X1_mfcc=extract_mfcc(path)
X1 = [x for x in X1_mfcc]
X1 = np.array(X1)
X1 = np.expand_dims(X1, -1)
X1=X1.reshape((1,X1.shape[0],X1.shape[1]))

In [269]:
predict_y1=model.predict(X1) 
yhat1=np.argmax(predict_y1,axis=1)

In [273]:
print(yhat1)

In [272]:
from tabulate import tabulate

mydata = [
    ["angry", "0"],
    ["disgust", "1"],
    ["fear", "2"],
    ["happy", "3"],
    ["neutral","4"],
    ["ps","5"],
    ["sad","6"]
]

head = ["Classes", "Encoded_value"]

# display table
print(tabulate(mydata, headers=head, tablefmt="grid"))